In [2]:
import numpy as np
from sympy import *
import matplotlib.pyplot as plt
%matplotlib inline
from celmech.nbody_simulation_utilities import set_time_step,align_simulation
from celmech.nbody_simulation_utilities import get_simarchive_integration_results
import rebound as rb
from os.path import isfile
from sympy import init_printing
init_printing()

In [3]:
from celmech.secular import LaplaceLagrangeSystem
from celmech.poincare import Poincare

In [67]:
def eccMatrix(alpha, mass):
    mu1, mu2, mu3 = mass
    alpha12, alpha23 = alpha
    P1, P2, P3 = alpha12**(3/2), 1, 1/alpha23**(3/2)

    sim = rb.Simulation()
    sim.add(m=1.)
    sim.add(m=mu1, P=P1)
    sim.add(m=mu2, P=P2)
    sim.add(m=mu3, P=P3)
    ps = sim.particles
    ps[1].r = ps[1].a*(ps[1].m/3/ps[0].m)**(1/3)
    ps[2].r = ps[2].a*(ps[2].m/3/ps[0].m)**(1/3)

    sim.move_to_com()
    sim.integrator = "whfast"
    sim.dt = sim.particles[1].P/20
    sim.collision = "direct"
    align_simulation(sim)
    
    
#     pvars = Poincare.from_Simulation(sim)
#     print((pvars.particles[1].a+pvars.particles[2].a+pvars.particles[3].a)/3)
#     alpha12 = pvars.particles[1].a/pvars.particles[2].a
#     alpha23 = pvars.particles[2].a/pvars.particles[3].a
#     alpha13 = pvars.particles[1].a/pvars.particles[3].a
#     print(f"alpha = {alpha12, alpha23, alpha13}")

    lsys = LaplaceLagrangeSystem.from_Simulation(sim)  
    M = lsys.Neccentricity_matrix
#     print(f"eccentricity matrix =")
#     print(M)
    
    return M

In [79]:
def eccMatrix_diag(alpha, mass):
    m1, m2, m3 = mass
    m_tot = m1+m2+m3
    alpha12, alpha23 = alpha
    alpha13 = alpha12*alpha23
    
    R2 = np.array([[np.sqrt(m1/m_tot), np.sqrt(m2/m_tot), np.sqrt(m3/m_tot)],
             [-np.sqrt(m3/(m1+m3)), 0, np.sqrt(m1/(m1+m3))],
             [np.sqrt(m1*m2/(m1+m3)/m_tot), -np.sqrt((m1+m3)/m_tot), np.sqrt(m2*m3/(m1+m3)/m_tot)]])
    M = eccMatrix(alpha, mass)
    
    vals,T = np.linalg.eigh(M) # T returned by eigh is R^T in paper. Could define R = T.T
    print(T.T)
    
#     print(np.matmul(np.matmul(R2, M), np.transpose(R2)))
    
# #     R1 = np.array([[1, 0, 0],
# #                 [0, np.cos(phi), np.sin(phi)],
# #                 [0, -np.sin(phi), np.cos(phi)]])
# #     R = np.matmul(R1, R2)
    
    vals, k = np.linalg.eigh(M)
    
#     return np.matmul(np.matmul(R, M), np.transpose(R))

In [80]:
x = eccMatrix_diag((0.85,0.85), (3e-6,1e-100,3e-6))
# print(x)
# print(f"sine = {np.sin(psi)}, cosine = {np.cos(psi)}")
# print(f"sine = {k/delta}, cosine = {np.sqrt(1-(k/delta)**2)}")

[[ 0.00000000e+00 -1.00000000e+00  0.00000000e+00]
 [-7.40719746e-01  0.00000000e+00  6.71814154e-01]
 [ 6.71814154e-01 -1.11022302e-16  7.40719746e-01]]
